In [61]:
!apt-get update -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [62]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [63]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [64]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [65]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [66]:
!pip install -q findspark
import findspark
findspark.init()

In [67]:
from pyspark.sql import SparkSession
spark = (SparkSession
.builder
.appName("<app_name>")
.getOrCreate())

In [68]:
from google.colab import drive

In [69]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
df_history=spark.read.csv('/content/drive/MyDrive/For_DG/chess_wc_history_game_info.csv',header='true')
df_moves=spark.read.csv('/content/drive/MyDrive/For_DG/chess_wc_history_moves.csv',header='true')
df_eco=spark.read.csv('/content/drive/MyDrive/For_DG/eco_codes.csv',header='true')

In [71]:
df_history.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            dra

In [72]:
df_moves.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [73]:
df_eco.show()

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
|A10|             English|                1 c4|       A|    Flank|
|A11|English, Caro-Kan...|             1 c4 c6|       A|    Fl

In [74]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window


**1.List of Winners of Each World champions Trophy**

Hint: Total Result of all rounds of Tournament for that player is considered as that player's Score/Result.

Result attributes: winner, tournament_name


In [75]:
res = df_history.where((df_history.tournament_name.contains('WorldChamp')) & ~df_history.winner.isin(['draw']))
column = ['winner','tournament_name']
res = res.select(column).groupBy(column).count()
result = Window.partitionBy("tournament_name").orderBy(F.col('count').desc())
res = res.withColumn('countrow',F.row_number().over(result)).where(F.col('countrow') == 1)
data_1 = ['count,countrow']
res = res.orderBy(F.col('tournament_name')).drop(*data_1)
res1 = res.select(column)
res1.show()
res1=res1.toPandas()
res1.head()
res1.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol1.csv')

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|   Steinitz, William| WorldChamp1886|
|   Steinitz, William| WorldChamp1889|
|   Steinitz, William| WorldChamp1890|
|   Steinitz, William| WorldChamp1892|
|     Lasker, Emanuel| WorldChamp1894|
|     Lasker, Emanuel| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1907|
|     Lasker, Emanuel| WorldChamp1908|
|     Lasker, Emanuel| WorldChamp1909|
|    Schlechter, Carl|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910b|
|Capablanca, Jose ...| WorldChamp1921|
| Alekhine, Alexander| WorldChamp1927|
| Alekhine, Alexander| WorldChamp1929|
| Alekhine, Alexander| WorldChamp1934|
|           Euwe, Max| WorldChamp1935|
| Alekhine, Alexander| WorldChamp1937|
|  Botvinnik, Mikhail| WorldChamp1948|
|  Botvinnik, Mikhail| WorldChamp1951|
|  Botvinnik, Mikhail| WorldChamp1954|
+--------------------+---------------+
only showing top 20 rows



**2.List of Players with number of times they have won Tournament in descending order(Max to min)**.

Result attributes: player_name, number_of_wins

In [76]:
from pyspark.sql.functions import col,asc,desc
win_count=df_history.where(df_history.winner!='draw').groupBy("winner").count()
res2=win_count.sort(col("count").desc())
res2.show()
res2=res2.toPandas()
res2.head()
res2.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol2.csv')

+--------------------+-----+
|              winner|count|
+--------------------+-----+
|     Lasker, Emanuel|   52|
|     Karpov, Anatoly|   47|
|  Botvinnik, Mikhail|   46|
|   Steinitz, William|   43|
| Alekhine, Alexander|   43|
|           Topalov,V|   32|
|      Kasparov, Gary|   31|
|             Anand,V|   30|
|    Smyslov, Vassily|   24|
|            Adams,Mi|   20|
|      Kasimdzhanov,R|   19|
|          Grischuk,A|   17|
|           Svidler,P|   16|
|      Adams, Michael|   16|
|            Bareev,E|   15|
|             Dreev,A|   15|
|            Shirov,A|   15|
|   Chigorin, Mikhail|   14|
|Khalifman, Alexander|   14|
|           Euwe, Max|   14|
+--------------------+-----+
only showing top 20 rows



**3.Most and Least Popular eco move in world championship history.**

Result attributes: eco, eco_name, number_of_occurences

Final result will have only two rows

In [77]:
dm=df_eco.select("eco","eco_name")
pop=dm.groupBy("eco_name").count()
pop=pop.sort(col("count").desc())
pop.registerTempTable("df_res")
res3=spark.sql("select eco_name, count from df_res where count = (select max(count) from df_res) union all select eco_name, count from df_res where count = (select min(count) from df_res)")
res3.show()
res3=res3.toPandas()
res3.head()
res3.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol3.csv')

+--------------------+-----+
|            eco_name|count|
+--------------------+-----+
|            Sicilian|   15|
|Sicilian, Acceler...|    1|
|       Three Knights|    1|
|Ruy Lopez, Bird's...|    1|
|King's Indian, Or...|    1|
|King's Indian Def...|    1|
|          Old Benoni|    1|
|    Sicilian, Alapin|    1|
|Sicilian, Dragon,...|    1|
|Dutch, Stonewall,...|    1|
|Queen's Gambit De...|    1|
|Catalan, Closed, ...|    1|
|King's Indian, Fo...|    1|
|Benoni, Classical...|    1|
|Sicilian, Richter...|    1|
|Sicilian, Fischer...|    1|
|King's Indian, Or...|    1|
|Benoni, Classical...|    1|
|   Caro-Kann Defense|    1|
|     French, Advance|    1|
+--------------------+-----+
only showing top 20 rows



**4.Find the eco move with most winnings.**

Ps. Use this opening move in your next chess game🙂

Result attributes: eco, eco_name

In [78]:
r1=df_history.select("eco")
popl=r1.groupBy("eco").count()
popl=popl.sort(col("count").desc())
r2=df_eco.select("eco","eco_name")
popl.registerTempTable("df_res1")
r2.registerTempTable("df_res2")
res4=spark.sql("select t2.eco,t2.eco_name from df_res1 t1,df_res2 t2 where t1.count=(select max(count) from df_res1) and t1.eco=t2.eco")
res4.show()
res4=res4.toPandas()
res4.head()
res4.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol4.csv')

+---+--------------+
|eco|      eco_name|
+---+--------------+
|C42|Petrov Defense|
+---+--------------+



**5.Longest and shortest game ever played in a world championship in terms of move.**

Chess Funda: "move" is completed once both White and Black have played one turn. e.g If a game lasts 10 moves, both White and Black have
played 10 moves)

Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [79]:
t1=df_moves.select("game_id","move_no_pair")
t2=df_history.select("game_id","tournament_name","event")

popular=t1.groupBy("game_id").count()
popular=popular.sort(col("count").desc())

popular.registerTempTable("df_table1")
t2.registerTempTable("df_table2")

res5=spark.sql("select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select max(count) from df_table1) and t1.game_id=t2.game_id union all select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1,df_table2 t2 where t1.count = (select min(count) from df_table1) and t1.game_id=t2.game_id")
res5.show()
res5=res5.toPandas()
res5.head()
res5.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol5.csv')

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|            291|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              1|
+--------------------+--------------------+---------------+---------------+



**6.Shortest and Longest Draw game ever Played.**

Result attributes: game_id, event, tournament_name, number_of_moves

Final result will have only two rows

In [80]:
t1=df_moves.select("game_id","move_no_pair")
t2=df_history.select("game_id","tournament_name","event","winner").where(df_history.winner=="draw")

popular=t1.groupBy("game_id").count()
popular=popular.sort(col("count").desc())

popular.registerTempTable("df_table1")
t2.registerTempTable("df_table2")

res6=spark.sql("select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select max(count) from df_table1 t1, df_table2 t2 where t2.game_id=t1.game_id) and t1.game_id=t2.game_id union all select t2.game_id,t2.event,t2.tournament_name,t1.count as number_of_moves from df_table1 t1, df_table2 t2 where t1.count = (select min(count) from df_table1 t1, df_table2 t2 where t2.game_id=t1.game_id) and t1.game_id=t2.game_id")
res6.show()

res6=res6.toPandas()
res6.head()
res6.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol6.csv')

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            258|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|             12|
+--------------------+-------------+---------------+---------------+



**7.Most and Least rated Player.**

Result attributes: player_name, elo

Chess Funda: elo is the rating of the player in chess tournament.

Final result will have only two rows

In [81]:
top=df_history.select("winner","winner_elo").where((df_history.winner!="null") & (df_history.winner!="draw")).distinct()
top_in=top.sort(col("winner_elo").desc())
top_in.registerTempTable("df_res1")
res7=spark.sql("select winner as player_name, winner_elo as elo from df_res1 where winner_elo = (select max(winner_elo) from df_res1) union all select winner as player_name, winner_elo as elo from df_res1 where winner_elo = (select min(winner_elo) from df_res1)")
res7.show()
res7=res7.toPandas()
res7.head()
res7.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol7.csv')

+---------------+----+
|    player_name| elo|
+---------------+----+
|Carlsen, Magnus|2870|
|     Ciuksyte,D|2271|
+---------------+----+



**8.3rd Last Player with most Loss.**

Result attributes: player_name

Final result will have only one row

In [82]:
runner=df_history.select("loser").where((df_history.loser!="null") & (df_history.loser!="draw"))
t1=runner.groupBy("loser").count()
t1=t1.sort(col("count").desc())
t1.registerTempTable("df_res")
res8=spark.sql("select loser from df_res limit 1")
res8.show()
res8=res8.toPandas()
res8.head()
res8.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol8.csv')

+-----------------+
|            loser|
+-----------------+
|Steinitz, William|
+-----------------+



**9.How many times players with low rating won matches with their total win Count.**

Result attributes: player_name, win_count

In [83]:
mwin=df_history.select("winner","winner_elo").where((df_history.winner!="null") & (df_history.winner!="draw") & (df_history.loser!="null") & (df_history.loser!="draw") & (df_history.loser_elo>df_history.winner_elo)).distinct()
mwin_r=mwin.sort(col("winner_elo").desc())
mwin_r.registerTempTable("df_res1")
res9=spark.sql("select winner as player_name, winner_elo as elo from df_res1 where winner_elo = (select max(winner_elo) from df_res1) union all select winner as player_name, winner_elo as elo from df_res1 where winner_elo = (select min(winner_elo) from df_res1)")
res9.show()
res9=res9.toPandas()
res9.head()
res9.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol9.csv')

+------------------+----+
|       player_name| elo|
+------------------+----+
|Anand, Viswanathan|2792|
|        Ciuksyte,D|2271|
+------------------+----+



**10.Move Sequence for Each Player in a Match.**

Result attributes: game_id, player_name, move_sequence, move_count

In [84]:
res1=df_moves.select("player","move_sequence","game_id")
res1.registerTempTable("df_result")
res10=spark.sql("select player,max(move_sequence) from df_result group by player")
res10.show()
res10=res10.toPandas()
res10.head()
res10.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol10.csv')

+------------------+--------------------+
|            player|  max(move_sequence)|
+------------------+--------------------+
| Hamdouchi, Hichem|f4|d5|Nf3|g6|d3|N...|
|   Alterman, Boris|e4|e5|Nf3|Nf6|Nc3...|
|   Fridman, Daniel|e4|e5|Bc4|Nf6|d3|...|
|    Svidler, Peter|e4|g6|d4|Bg7|Nc3|...|
|    Andersson, Ulf|e4|e6|d4|d5|e5|c5...|
|        Ivanchuk,V|e4|e6|d4|d5|e5|b6...|
|   Jussupow, Artur|e4|e5|Nf3|Nf6|Nc3...|
|  Nepomniachtchi,I|e4|e5|Nf3|Nf6|d4|...|
|          Piket,Je|e4|e5|Nf3|Nf6|d4|...|
|    Van Wely, Loek|e4|c5|Nf3|d6|c3|N...|
|       Onischuk,Al|e4|e5|Nf3|Nc6|Bb5...|
|      Peng Xiaomin|e4|e5|Nf3|Nc6|d4|...|
| Ivanchuk, Vassily|e4|e5|Nf3|Nc6|d4|...|
|    Kacheishvili,G|e4|c6|d4|d5|Nd2|d...|
|          Luther,T|e4|e6|d4|d5|Nd2|N...|
|Sakaev, Konstantin|d4|d5|c4|dxc4|e4|...|
| Steinitz, William|e4|e6|d4|d5|Nc3|d...|
|      Vladimirov,E|e4|e6|d4|d5|Nd2|N...|
|         Carlsen,M|e4|e5|Nf3|Nf6|d4|...|
|Istratescu, Andrei|e4|g6|d4|Bg7|c4|d...|
+------------------+--------------

**11.Total Number of games where losing player has more Captured score than Winning player.**

Hint: Captured score is cumulative, i.e., for 3rd capture it will have score 
for 1, 2, and 3rd.

Result attributes: total_number_of_games Final result will have only one row

**12.List All Perfect Tournament with Winner Name.**

Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7

Matches in tournament where 2 matches are draw and player B has won 0 matches)

Result attributes: winner_name, tournament_name

In [85]:
res1=df_history.select("game_id","tournament_name","event","winner","loser").where((df_history.winner!="draw")&(df_history.loser!="draw"))
res1.registerTempTable("df_result1")
res12=spark.sql("SELECT game_id,tournament_name,event,winner FROM df_result1 WHERE winner not in (SELECT distinct(loser) FROM df_result1)")
res12.show()
res12=res12.toPandas()
res12.head()
res12.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol12.csv')

+--------------------+---------------+-----------+--------------------+
|             game_id|tournament_name|      event|              winner|
+--------------------+---------------+-----------+--------------------+
|b528ad29-23c3-4bf...|  FideChamp2004|FIDE WCh KO|            Karlov,A|
|cd3774bf-05ac-4fb...|  FideChamp2002|FIDE WCh KO|          Ciuksyte,D|
|0b07b253-df45-41f...|  FideChamp2002|FIDE WCh KO|            Foisor,C|
|85c3014a-ce6c-45b...|  FideChamp2002|FIDE WCh KO|          Kiseleva,N|
|55dff081-1370-4d5...|  FideChamp2002|FIDE WCh KO|            Foisor,C|
|b8b237f1-40fa-44c...|  FideChamp2002|FIDE WCh KO|          Kiseleva,N|
|84228834-5a50-4b5...|  FideChamp2002|FIDE WCh KO|Stepovaia Dianche...|
|05e0250a-2550-4ea...|  FideChamp2002|FIDE WCh KO|       Zielinska,Mar|
+--------------------+---------------+-----------+--------------------+



**13.Player with highest winning ratio.**

Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)

Result attributes: player_name

Final result will have only one row

In [86]:
t1=df_history.select("winner").where(df_history.winner!="draw")
r1=t1.groupBy("winner").count()
t2=df_moves.select("player")
r2=t2.groupBy("player").count()
r1.registerTempTable("table1")
r2.registerTempTable("table2")

res13=spark.sql("select t1.winner as player_name from table1 t1,table2 t2 where t2.player=t1.winner and t1.count/t2.count==(select max(t1.count/t2.count) from table1 t1,table2 t2 where t2.player=t1.winner)")
res13.show()

res13=res13.toPandas()
res13.head()
res13.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol13.csv')

+-----------+
|player_name|
+-----------+
| Kiseleva,N|
+-----------+



**14.Player who had given checkmate with Pawn.**


Note: Consider all events for this query

Result attributes: player_name

Final result will have only one row

In [87]:
res14=df_moves.select("player").where((df_moves.piece=="P") & (df_moves.is_check_mate==1))
res14.show()
res14=res14.toPandas()
res14.head()
res14.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol14.csv')

+--------------+
|        player|
+--------------+
|Andersson, Ulf|
+--------------+



**15.List games where player has won game without queen.**


Result attributes: game_id, event, player_name

In [88]:
res15=df_moves.select(['game_id','player']).where((df_moves.white_queen_count==0) &(df_moves.black_queen_count==0) & (df_moves.is_check_mate==1))
res15.show()
res15=res15.toPandas()
res15.head()
res15.to_csv('/content/drive/MyDrive/DE_SOLUTION_DHANAVEL_K/results/' +'sol15.csv')

+--------------------+--------------+
|             game_id|        player|
+--------------------+--------------+
|04c81fe4-efd8-456...|    Nakamura,H|
|ac45a5ed-172d-4ae...|Andersson, Ulf|
+--------------------+--------------+

